In [1]:
from sagas.ofbiz.services import OfService as s, oc, track
from sagas.ofbiz.entities import OfEntity as e
from sagas.ofbiz.date_time import DateTime

e('meta').TestFieldType

,internal,name,primary,type,field type
0,,testFieldTypeId,*,id,id
1,,blobField,,blob,blob
2,,byteArrayField,,byte-array,byte-array
3,,objectField,,object,object
4,,dateField,,date,date
5,,timeField,,time,time
6,,dateTimeField,,date-time,date-time
7,,fixedPointField,,fixed-point,fixed-point
8,,floatingPointField,,floating-point,floating-point
9,,numericField,,numeric,numeric


In [2]:
type_mappings={"string":["object",
                         "id","id-long","id-vlong",
                         "indicator","very-short","short-varchar","long-varchar","very-long",
                         "comment","description","name","value",
                         "credit-card-number","credit-card-date","email","url","tel-number"],
              "float":["currency-amount","currency-precise","fixed-point","floating-point"],
              "date":["date-time","date"],
              "time":["time"],
              "bytes":["byte-array","blob"],
              "int":["numeric"]}
gl_mappings={"string":"String", 
             "float":"double", "int":"int",
             "date":"DateTime",
             "time":"DateTime",
             "bytes":"Uint8List"
            }

def get_mapping_type(field_type):
    for k, v in type_mappings.items():
        if field_type in v:
            return k
    raise ValueError("Cannot find mapping type for "+field_type)
    
def get_dart_type(field_type):
    mt=get_mapping_type(field_type)
    return gl_mappings[mt]

In [57]:
print("%st%s"%('hello','world'))

hellotworld


In [16]:
import clipboard

package_header='''import 'dart:typed_data';
import 'package:meta/meta.dart';
import 'package:sagas_meta/src/entity_base.dart';'''
entity_header='''
/// Entity {entity}, {title}
class {entity} extends EntityBase {{
'''
entity_footer='''    lastUpdatedStamp,
    createdStamp}) : super(entityId, lastUpdatedStamp, createdStamp);

  @override
  String toString() => '%s { id: $entityId }';
'''
entity_to_map='''  @override
  Map<String, dynamic> asMap() {
    return {%s};
  }
'''
end_mark="}"

lines=[]
lines.append(package_header)

def gen_bloc_model(lines, entity_name):
    ent = oc.delegator.getModelEntity(entity_name)
    lines.append(entity_header.format(
        entity=entity_name,
        title=ent.getTitle()
    ))
    pk_size=ent.getPksSize()
    if pk_size==1:
        lines.append("  /// this entity has only one pk")
    names = ent.getAllFieldNames()
    field_setters=[]
    field_map=[]
    for field_name in names:
        fld = ent.getField(field_name)
        if not fld.getIsAutoCreatedInternal():
            fldtype=get_dart_type(fld.getType())
            require_mark=""
            comment=""
            if fld.getIsPk():
                require_mark="@required "
                comment=" // pk"
            field_setters.append('%sthis.%s'%(require_mark, field_name))
            field_map.append("'{name}':{name}".format(name=field_name))
            lines.append('  final {type} {name};{comment}'.format(
                name=field_name, 
                comment=comment,
                type=fldtype))

    lines.append('  %s({entityId,'%entity_name)
    lines.append('    %s,'%', '.join(field_setters))
    lines.append(entity_footer%entity_name)
    lines.append(entity_to_map%', '.join(field_map))
    lines.append(end_mark)

entities=['Testing', 'TestingType', 'TestFieldType', 'Person']
for entity_name in entities:
    # entity_name='TestFieldType'
    gen_bloc_model(lines, entity_name)

print("\n".join(lines))
clipboard.copy("\n".join(lines))

import 'dart:typed_data';
import 'package:meta/meta.dart';
import 'package:sagas_meta/src/entity_base.dart';

/// Entity Testing, Testing
class Testing extends EntityBase {

  /// this entity has only one pk
  final String comments;
  final String testingTypeId;
  final int testingSize;
  final String testingId; // pk
  final String description;
  final DateTime testingDate;
  final String testingName;
  Testing({entityId,
    this.comments, this.testingTypeId, this.testingSize, @required this.testingId, this.description, this.testingDate, this.testingName,
    lastUpdatedStamp,
    createdStamp}) : super(entityId, lastUpdatedStamp, createdStamp);

  @override
  String toString() => 'Testing { id: $entityId }';

  @override
  Map<String, dynamic> asMap() {
    return {'comments':comments, 'testingTypeId':testingTypeId, 'testingSize':testingSize, 'testingId':testingId, 'description':description, 'testingDate':testingDate, 'testingName':testingName};
  }

}

/// Entity TestingType, Testi

In [4]:
def get_id_list(entity_name):
    ent = oc.delegator.getModelEntity(entity_name)
    names=ent.getPkFieldNames()
    flds=[]
    for pk in names:
        flds.append("'%s'"%pk)
    return(', '.join(flds))    

In [17]:
from sagas.util.str_converters import to_camel_case, to_snake_case

imports='''import 'package:sagas_meta/src/entity_base.dart';
import 'package:sagas_meta/src/models_t/gen_models.dart';
import 'dart:convert';
import 'dart:typed_data';
import 'package:intl/intl.dart';
'''
default_fields='''        lastUpdatedStamp: check_dt(json['lastUpdatedStamp']),
        createdStamp: check_dt(json['createdStamp']));'''
id_field='''        entityId: create_id_from('{entity}', [{id_list}], json),'''
package_name='testing'
lines=[]
lines.append(imports)
lines.append('class %sEntityJsonifier{'%to_camel_case(package_name, True))

entity_json_header='''  static {entity} extract{entity}(dynamic json) {{
    return {entity}('''
field_def='''        {name}: json['{name}'] as {type},'''
date_time_def='''        {name}: check_dt(json['{name}']),'''
time_def='''        {name}: check_time(json['{name}']),'''
bytes_def='''        {name}: check_b64(json['{name}']),'''
field_setter_mappings={"string":field_def, 
                     "float":field_def, 
                     "int":field_def,
                     "date":date_time_def,
                     "time":time_def,
                     "bytes":bytes_def
                    }

def gen_bloc_jsonifier(lines, entity_name):
    ent = oc.delegator.getModelEntity(entity_name)
    lines.append(entity_json_header.format(entity=entity_name))
    lines.append(id_field.format(entity=entity_name, id_list=get_id_list(entity_name)))
    names = ent.getAllFieldNames()
    field_setters=[]
    for field_name in names:
        fld = ent.getField(field_name)
        if not fld.getIsAutoCreatedInternal():
            fldtype=get_dart_type(fld.getType())
            mapping_type=get_mapping_type(fld.getType())
            setter=field_setter_mappings[mapping_type]
            # print()
            lines.append(setter.format(name=field_name, type=fldtype))
    lines.append(default_fields)
    lines.append('  }\n')

entities=['Testing', 'TestingType', 'TestFieldType', 'Person']
for entity in entities:
    gen_bloc_jsonifier(lines, entity)

# end package
lines.append('}')
print("\n".join(lines))
clipboard.copy("\n".join(lines))

import 'package:sagas_meta/src/entity_base.dart';
import 'package:sagas_meta/src/models_t/gen_models.dart';
import 'dart:convert';
import 'dart:typed_data';
import 'package:intl/intl.dart';

class TestingEntityJsonifier{
  static Testing extractTesting(dynamic json) {
    return Testing(
        entityId: create_id_from('Testing', ['testingId'], json),
        comments: json['comments'] as String,
        testingTypeId: json['testingTypeId'] as String,
        testingSize: json['testingSize'] as int,
        testingId: json['testingId'] as String,
        description: json['description'] as String,
        testingDate: check_dt(json['testingDate']),
        testingName: json['testingName'] as String,
        lastUpdatedStamp: check_dt(json['lastUpdatedStamp']),
        createdStamp: check_dt(json['createdStamp']));
  }

  static TestingType extractTestingType(dynamic json) {
    return TestingType(
        entityId: create_id_from('TestingType', ['testingTypeId'], json),
        testin